In [33]:
"""
Data cleaning and preprocessing script for Airbnb dataset analysis.
This script handles three main datasets: hosts, reviews, and rooms.
"""

import pandas as pd
import numpy as np
import locale

In [34]:
# Load main datasets
DATA_PATH = "../data"
hosts_df = pd.read_csv(f"{DATA_PATH}/hosts.csv", sep=',')
reviews_df = pd.read_csv(f"{DATA_PATH}/reviews.csv", sep=',')

In [35]:
# Load rooms dataset with specific parsing parameters
rooms_df = pd.read_csv(
    f"{DATA_PATH}/rooms.csv",
    sep=",",
    quotechar='"',
    encoding="utf-8",
    on_bad_lines="skip"  # Skip problematic rows
)

In [36]:
def analyze_dataframe_quality(df: pd.DataFrame) -> None:
    """
    Analyze basic quality metrics of a dataframe.
    
    Args:
        df: pandas DataFrame to analyze
    """
    print(f"Shape: {df.shape}")
    print("\nMissing Values:")
    display(df.isnull().sum())
    print("\nDuplicate Records:")
    duplicate_count = len(df[df.duplicated()])
    print(f"Total duplicate records: {duplicate_count}")

In [37]:
def analyze_categorical_columns(df: pd.DataFrame) -> None:
    """
    Analyze categorical columns in a dataframe.
    
    Args:
        df: pandas DataFrame to analyze
    """
    # Display basic information about the dataframe
    display(df.info())
    
    # Standardize column names
    df.columns = df.columns.str.lower()
    
    # Process categorical columns
    categorical_cols = df.select_dtypes(include='object').columns
    
    # Clean and analyze each categorical column
    for col in categorical_cols:
        # Clean string values
        df[col] = df[col].str.strip().str.lower()
        
        # Analyze unique values
        unique_values = df[col].unique()
        print(f"\nColumn: {col} | Unique values: {len(unique_values)}")
        
        # Display unique values if less than 20 categories
        if len(unique_values) < 20:
            print("Categories:", unique_values)

In [38]:
def analyze_numerical_columns(df: pd.DataFrame) -> dict:
    """
    Analyze numerical columns for inconsistencies and outliers.
    
    Args:
        df: pandas DataFrame to analyze
    
    Returns:
        dict: Dictionary containing outlier counts per column
    """
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    outliers_summary = {}
    
    # Check for invalid numerical values
    for col in numerical_cols:
        invalid_mask = ~df[col].apply(lambda x: np.isreal(x)) & df[col].notnull()
        invalid_records = df[invalid_mask]
        
        if not invalid_records.empty:
            print(f"Invalid values found in {col}:")
            display(invalid_records)
    
    # Detect outliers using IQR method
    for col in numerical_cols:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outliers_summary[col] = len(outliers)
        print(f"{col}: {len(outliers)} outliers detected")
    
    return outliers_summary

### Hosts

In [39]:
analyze_dataframe_quality(hosts_df)

Shape: (37484, 2)

Missing Values:


host_id       0
host_name    18
dtype: int64


Duplicate Records:
Total duplicate records: 0


In [40]:
hosts_df = hosts_df.dropna()

In [41]:
analyze_categorical_columns(hosts_df)

<class 'pandas.core.frame.DataFrame'>
Index: 37466 entries, 0 to 37483
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   host_id    37466 non-null  object
 1   host_name  37466 non-null  object
dtypes: object(2)
memory usage: 878.1+ KB


None


Column: host_id | Unique values: 37461

Column: host_name | Unique values: 11399


In [42]:
# Corrigindo id
mask_hosts = pd.to_numeric(hosts_df['host_id'], errors='coerce').isna()
hosts_df = hosts_df[~mask_hosts].copy()
hosts_df['host_id'] = pd.to_numeric(hosts_df['host_id'])

### Reviews

In [43]:
analyze_dataframe_quality(reviews_df)

Shape: (48875, 8)

Missing Values:


id                                    0
host_id                               0
price                                 0
number_of_reviews                    20
last_review                       10039
reviews_per_month                 10019
calculated_host_listings_count        0
availability_365                    156
dtype: int64


Duplicate Records:
Total duplicate records: 0


In [44]:
# Remover linhas com valores nulos em 'price' e 'availability_365'
reviews_df = reviews_df.dropna(subset=['price', 'availability_365'])

# Substituir valores nulos em 'last_review' e 'reviews_per_month' por -1
reviews_df['reviews_per_month'] = reviews_df['reviews_per_month'].fillna(-1)

In [45]:
analyze_categorical_columns(reviews_df)

<class 'pandas.core.frame.DataFrame'>
Index: 48719 entries, 0 to 48874
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48719 non-null  object 
 1   host_id                         48719 non-null  object 
 2   price                           48719 non-null  int64  
 3   number_of_reviews               48719 non-null  object 
 4   last_review                     38700 non-null  object 
 5   reviews_per_month               48719 non-null  float64
 6   calculated_host_listings_count  48719 non-null  int64  
 7   availability_365                48719 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 3.3+ MB


None


Column: id | Unique values: 48719

Column: host_id | Unique values: 37347

Column: number_of_reviews | Unique values: 394

Column: last_review | Unique values: 1765


In [46]:
# number_of_reviews: object -> int
reviews_df['number_of_reviews'] = reviews_df['number_of_reviews'].astype(int)

# availability_365: float -> int
reviews_df['availability_365'] = reviews_df['availability_365'].astype(int)

# last_review: object -> datatime
reviews_df['last_review'] = pd.to_datetime(reviews_df['last_review'], errors='coerce')
reviews_df['last_review'] = reviews_df['last_review'].fillna(-1)

In [47]:
# Corrigindo id
mask_reviews_df = pd.to_numeric(reviews_df['id'], errors='coerce').isna()
reviews_df = reviews_df[~mask_reviews_df].copy()
reviews_df['id'] = pd.to_numeric(reviews_df['id'])

# host_id: object -> int
reviews_df['host_id'] = reviews_df['host_id'].astype(int)

In [48]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48710 entries, 0 to 48874
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48710 non-null  int64  
 1   host_id                         48710 non-null  int64  
 2   price                           48710 non-null  int64  
 3   number_of_reviews               48710 non-null  int64  
 4   last_review                     48710 non-null  object 
 5   reviews_per_month               48710 non-null  float64
 6   calculated_host_listings_count  48710 non-null  int64  
 7   availability_365                48710 non-null  int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 3.3+ MB


In [49]:
analyze_numerical_columns(reviews_df)

id: 0 outliers detected
host_id: 1514 outliers detected
price: 2962 outliers detected
number_of_reviews: 6098 outliers detected
reviews_per_month: 3288 outliers detected
calculated_host_listings_count: 7055 outliers detected
availability_365: 0 outliers detected


{'id': 0,
 'host_id': 1514,
 'price': 2962,
 'number_of_reviews': 6098,
 'reviews_per_month': 3288,
 'calculated_host_listings_count': 7055,
 'availability_365': 0}

### Rooms

In [50]:
analyze_dataframe_quality(rooms_df)

Shape: (48875, 8)

Missing Values:


id                     0
name                   0
neighbourhood          0
neighbourhood_group    0
latitude               0
longitude              0
room_type              0
minimum_nights         0
dtype: int64


Duplicate Records:
Total duplicate records: 0


In [51]:
analyze_categorical_columns(rooms_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48875 entries, 0 to 48874
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   48875 non-null  object 
 1   name                 48875 non-null  object 
 2   neighbourhood        48875 non-null  object 
 3   neighbourhood_group  48875 non-null  object 
 4   latitude             48875 non-null  float64
 5   longitude            48875 non-null  object 
 6   room_type            48875 non-null  object 
 7   minimum_nights       48875 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 3.0+ MB


None


Column: id | Unique values: 48862

Column: name | Unique values: 47437

Column: neighbourhood | Unique values: 377

Column: neighbourhood_group | Unique values: 68

Column: longitude | Unique values: 14686

Column: room_type | Unique values: 77


In [52]:
# Converter colunas para os tipos corretos
rooms_df['id'] = pd.to_numeric(rooms_df['id'], errors='coerce').astype('Int64')
rooms_df['minimum_nights'] = pd.to_numeric(rooms_df['minimum_nights'], errors='coerce').astype('Int64')

# Remover espaços extras em colunas de texto
for col in ['name', 'neighbourhood', 'neighbourhood_group', 'room_type']:
    rooms_df[col] = rooms_df[col].astype(str).str.strip()

In [53]:
rooms_df = rooms_df.dropna()

### Export

In [54]:
def export_for_power_bi(df: pd.DataFrame, output_path: str) -> None:
    """
    Export DataFrame to CSV with proper decimal formatting for Power BI.
    
    Args:
        df: pandas DataFrame to export
        output_path: path where the CSV file will be saved
    """
    # Set system locale for decimal separator detection
    locale.setlocale(locale.LC_ALL, '')
    decimal_separator = locale.localeconv()['decimal_point']
    
    # Create copy for export
    df_export = df.copy()
    
    # Handle decimal separator for geographical coordinates
    if decimal_separator == ',':
        for col in ['latitude', 'longitude']:
            if col in df_export.columns:
                df_export[col] = df_export[col].astype(str).str.replace('.', ',')
    
    # Export with proper encoding
    df_export.to_csv(
        output_path,
        index=False,
        encoding='utf-8',
        sep=';'  # Standard separator for Excel in non-English locales
    )
    print(f"File exported successfully: {output_path}")

In [55]:
# Export cleaned datasets
CLEAN_DATA_PATH = "../data/clean_data"
export_for_power_bi(hosts_df, f"{CLEAN_DATA_PATH}/hosts_clean.csv")
export_for_power_bi(reviews_df, f"{CLEAN_DATA_PATH}/reviews_clean.csv")
export_for_power_bi(rooms_df, f"{CLEAN_DATA_PATH}/rooms_clean.csv")

File exported successfully: ../data/clean_data/hosts_clean.csv
File exported successfully: ../data/clean_data/reviews_clean.csv
File exported successfully: ../data/clean_data/rooms_clean.csv
